# **Imports**

In [1]:
import os
os.chdir(os.path.expanduser("~/PycharmProjects/vivarium-ecoli"))

In [2]:
import argparse
import numpy as np
import pathlib, datetime
import pandas as pd
import math
import seaborn as sns

# vivarium-core imports
import pytest
from vivarium.core.engine import Engine
from vivarium.core.composer import Composer
from vivarium.library.dict_utils import deep_merge

# vivarium-ecoli imports
from ecoli.experiments.ecoli_master_sim import EcoliSim, CONFIG_DIR_PATH
from ecoli.library.sim_data import LoadSimData
from ecoli.states.wcecoli_state import get_state_from_file
from ecoli.composites.ecoli_master import SIM_DATA_PATH
from ecoli.processes.registries import topology_registry

# **Parameters**

In [3]:
filename='default'
total_time=60
divide=True
initial_state_file='vivecoli_t2'
progress_bar=True
log_updates=False
emitter='timeseries'

In [4]:
sim = EcoliSim.from_file(CONFIG_DIR_PATH + filename + '.json')
sim.total_time = total_time
sim.raw_output = False
sim.divide = divide
sim.progress_bar = progress_bar
sim.log_updates = log_updates
sim.emitter = emitter
sim.initial_state = get_state_from_file(path=f'data/{initial_state_file}.json')

In [5]:
sim.build_ecoli()

In [6]:
sim.processes

{'ecoli-chromosome-structure': ecoli.processes.chromosome_structure.ChromosomeStructure,
 'ecoli-metabolism': ecoli.processes.metabolism.Metabolism,
 'ecoli-tf-binding': ecoli.processes.tf_binding.TfBinding,
 'ecoli-transcript-initiation': ecoli.processes.transcript_initiation.TranscriptInitiation,
 'ecoli-transcript-elongation': ecoli.processes.transcript_elongation.TranscriptElongation,
 'ecoli-rna-degradation': ecoli.processes.rna_degradation.RnaDegradation,
 'ecoli-polypeptide-initiation': ecoli.processes.polypeptide_initiation.PolypeptideInitiation,
 'ecoli-polypeptide-elongation': ecoli.processes.polypeptide_elongation.PolypeptideElongation,
 'ecoli-complexation': ecoli.processes.complexation.Complexation,
 'ecoli-two-component-system': ecoli.processes.two_component_system.TwoComponentSystem,
 'ecoli-equilibrium': ecoli.processes.equilibrium.Equilibrium,
 'ecoli-protein-degradation': ecoli.processes.protein_degradation.ProteinDegradation,
 'ecoli-chromosome-replication': ecoli.pr

# **Run Simulation**

In [7]:
sim.run()


Simulation ID: 88c20826-5efc-11ed-9d41-acde48001122
Created: 11/07/2022 at 16:30:27
Progress:|██████████████████████████████████████████████████| 0.0/60.0 simulated seconds remaining     
Completed in 79.14 seconds


# **Query Output**

In [8]:
folderName = input("Enter the output Folder Path. If prefer default, press ENTER:")

if folderName == "":
    folder = f'out/geneRxnVerifData'
else:
    folder = f'{folderName}'

Enter the output Folder Path. If prefer default, press ENTER: 


# *Enter Query Parameters*

In [ ]:
query = []
agents = sim.query()['agents'].keys()
for agent in agents:
    query.extend([('agents', agent, 'listeners', 'fba_results'),
                  ('agents', agent, 'bulk'), ('agents', agent, 'listeners', 'mass')])
output = sim.query(query)

pathlib.Path(folder).mkdir(parents=True, exist_ok=True)

# *Visualize Output*

## Basic Visualization

In [ ]:
all_masses = output['agents']['0']['listeners']['mass']
all_masses = pd.DataFrame(all_masses)

In [ ]:
all_masses

### Masses to be visualized contained in array below

In [ ]:
collect = ['cell_mass', 'dry_mass', 'dnaMass', 'proteinMass', 'water_mass', 'smallMoleculeMass']

In [ ]:
masses = all_masses.loc[:,collect]

In [ ]:
for i, name in enumerate(collect):
    maxVal = max(list(masses[name]))
    masses[name] = masses[name].div(maxVal)

In [ ]:
masses

In [ ]:
sns.lineplot(data=masses)

### For more advanced visualization, utilize other notebooks